In [1]:
# imports
import pandas as pd
import numpy as nd
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
num_key_words = 5
ncm_representativa = 3
categoria = 20

In [4]:
# carga arquivo
arquivo = "PROD_BEBIDAS.csv"
prod = pd.read_csv(arquivo, sep= ';', quotechar='"', encoding='latin1', usecols=[0,1,2,3,4])

In [5]:
prod.head()

,CATEGORIA,COD_PRODUTO,DES_PRODUTO,COD_NCM,MARCA
0,20,2024724001,CONHAQUE REMY MARTIN 100CL,22082000.0,REMY MARTIN
1,20,2031240001,"COURVOISIER VSOP, CONHAQUE AGUARDENTE DA DESTI...",22082000.0,COURVOISIER
2,20,2031259001,CONHAQUE MARTELL VSOP 40% 1L,22082000.0,MARTELL
3,20,2034010004,MARTELL XO 40% 70CL COGNAC,22089000.0,MARTELL
4,20,2039594001,CONHAQUE MARTELL VS FINE CHAMP 1L 40% 1L,22082000.0,MARTELL


In [ ]:
# removendo linhas de prod que não tenham descricao
#prod = prod.drop(prod[ not prod['DES_PRODUTO'].str.lower().str.islower()].index)
#x = [p for p in prod.DES_PRODUTO if not p.lower().islower() ]


In [6]:
# filtrando categoria a ser analisada removendo linhas que não tenham codigo produto
prod = prod[prod["CATEGORIA"] == categoria]
prod = prod[prod["COD_PRODUTO"].notna()]

In [7]:
prod["DES_PRODUTO"] = prod["DES_PRODUTO"].fillna("NA")
type(prod["DES_PRODUTO"])

pandas.core.series.Series

In [8]:
# buscando distintas marcas
marcas = prod["MARCA"].str.lower().dropna()
marcas = list(marcas.unique()) 

In [9]:
marcas

['remy martin',
 'courvoisier',
 'martell',
 'hennessy',
 'beluga',
 'aultmore',
 "grant's",
 'st-germain',
 'danzka',
 'licor 43',
 'stolichnaya',
 'johnnie walker',
 'glenlivet',
 'grey goose',
 'zacapa',
 'crystal head',
 'craigellachie',
 'royal brackla',
 'amazzoni',
 'amarula',
 'santa teresa',
 'durbanville hills',
 'antinori',
 'drambuie',
 'calastri',
 'calvet',
 'mozart',
 'nederburg',
 'costa di bussia',
 'bellussi',
 'cointreau',
 'beefeater',
 'santo grau',
 'weber haus',
 'araxa',
 'rose piscine',
 'cachaca pitu',
 'brasiliana',
 'leblon',
 'ypioca',
 'busca vida',
 'mata velha',
 'bruichladdich',
 'balpoggio',
 'santa vitoria',
 'ketel one',
 'versatil',
 'inevitavel',
 'adriano ramos-pinto',
 'absolut',
 'patron',
 'tarucco',
 'taittinger',
 'terrale',
 'saint felicien',
 'monkey shoulder',
 'sailor jerry',
 "jack daniel's",
 'anthonij rupert',
 'michel lynch',
 'torres',
 'ciroc',
 "graham's",
 'alessandro di campor',
 'bayou rum',
 'veuve clicquot',
 'le marchesine',


In [10]:
# lista de descrições de produtos em miniscula e sem NA
descricoes = prod["DES_PRODUTO"].str.lower()
type(descricoes)

pandas.core.series.Series

In [11]:
descricoes

0                              conhaque remy martin 100cl
1       courvoisier vsop, conhaque aguardente da desti...
2                            conhaque martell vsop 40% 1l
3                              martell xo 40% 70cl cognac
4                conhaque martell vs fine champ 1l 40% 1l
                              ...                        
3328                                                   na
3329                                                   na
3330                                                   na
3331                                                   na
3332                                                   na
Name: DES_PRODUTO, Length: 3333, dtype: object

In [12]:
# abre arquivo palavras Gol por NCM
gol = pd.read_excel("palavra GOL.xlsx")
gol = gol[gol["CATEGORIA"] == categoria]
gol

,CATEGORIA,PALAVRA GOL,NCM
12,20,vinho,22042100
13,20,champagne,22041010
14,20,champanhe,22041010
15,20,espumante,22041090
16,20,conhaque,22082000
17,20,cognac,22082000
18,20,uisque,22083020
19,20,whisky,22083020
20,20,rum,22084000
21,20,cachaca,22084000


In [13]:
# lista palavras que devem ser desconsideradas na analise
excluir = pd.read_excel("excluir_palavras.xlsx")
excluir = list(excluir["EXCLUIR"])
excluir

['contendo',
 'unidades',
 'unidade',
 'unid',
 'ades',
 'grama',
 'cigarros',
 'charutos',
 'contend',
 'unduiades',
 'filtros',
 'gramas',
 'tamanho',
 'seu',
 'cont',
 'endo',
 'com',
 'car',
 'biscoitos',
 'balas',
 'alcool',
 'alcoolico',
 'alcoolica',
 'teor',
 'garrafa',
 'garrafas',
 'cento',
 'cent',
 'ento',
 'apresentado',
 'apresentada',
 'bebida',
 'degusta',
 'degustacao',
 'tasting',
 'pacote',
 'cote',
 'para',
 'cada']

In [14]:
# lista distintas palavras das descrições que serão usadas na analise
padrao = '(?u)\\b[^\\d\\W]+\\b'
vet = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(descricoes)
t = vet.transform(descricoes)
palavras = vet.get_feature_names()
palavras = [p for p in palavras if (len(p) > 3 and p not in excluir and p not in marcas) ]
#desc = pd.DataFrame(t.todense(), columns = palavras)
palavras

['abacaxi',
 'absolute',
 'acai',
 'accord',
 'achocolatado',
 'acino',
 'acipestres',
 'aciprestes',
 'acquavita',
 'acrilico',
 'acucar',
 'adicao',
 'adriano',
 'adventurer',
 'africano',
 'agassac',
 'agave',
 'aged',
 'aguardente',
 'alandra',
 'alba',
 'albarino',
 'alcolica',
 'alcolico',
 'alcoolicas',
 'alcoolida',
 'alcoolido',
 'alentejano',
 'alentejo',
 'alessandro',
 'alexandre',
 'algodao',
 'alicante',
 'allora',
 'allure',
 'almaviva',
 'aloolica',
 'aloolico',
 'alta',
 'aluminio',
 'amado',
 'amaretto',
 'amarguinha',
 'amarone',
 'amassador',
 'amazonia',
 'amburana',
 'amendoa',
 'america',
 'american',
 'americano',
 'amostra',
 'amour',
 'anciente',
 'anderra',
 'andes',
 'andy',
 'anejo',
 'angelica',
 'anjou',
 'anna',
 'anos',
 'anthonij',
 'antonia',
 'apalta',
 'apck',
 'apeach',
 'aperitivo',
 'apertivo',
 'apimentado',
 'apple',
 'apresentadas',
 'apresentados',
 'apresentas',
 'apresnetado',
 'aprsentada',
 'apry',
 'arabico',
 'aragones',
 'araucano',
 '

In [ ]:
# lista de distintos NCMs
#ncm = prod["COD_NCM"].dropna().unique()
#ncm
# substituido pelo codigo abaixo

In [15]:
# Representatividade por NCM: quantidade de produtos por NCM
rep = prod[prod["DES_PRODUTO"] != 'NA']
rep = rep.groupby(["COD_NCM"]).count()["COD_PRODUTO"].rename("QTDE_PRODUTOS")
rep = rep[rep.notna()]
rep = rep[rep.values > ncm_representativa]
rep = rep.sort_values( ascending=False)
rep

COD_NCM
22042100.0    918
22083020.0    484
22089000.0    295
22041090.0    183
22087000.0    162
22084000.0    131
22086000.0     98
22041010.0     74
22083090.0     73
70132800.0     53
22083010.0     47
22085000.0     35
74032100.0     34
22082000.0     26
22051000.0     24
Name: QTDE_PRODUTOS, dtype: int64

In [16]:
ncm = nd.array(rep.index)
ncm

array([22042100., 22083020., 22089000., 22041090., 22087000., 22084000.,
       22086000., 22041010., 22083090., 70132800., 22083010., 22085000.,
       74032100., 22082000., 22051000.])

In [17]:
# gera df freq_df com quantidade de ocorrencias de cada palavras em cada NCM
ncm_fre = {}
for n in ncm:
    qtd = []
    #temp = prod.query('COD_NCM == ' + str(n))["DES_PRODUTO"]
    temp = prod[prod["COD_NCM"] == n]["DES_PRODUTO"].str.lower().dropna()
    for p in palavras: 
        freq = 0
        for d in temp : 
            freq += d.count(p)
        qtd.append(freq) 
    ncm_fre[n] = qtd 

freq_df = pd.DataFrame(ncm_fre, index = palavras, columns = ncm)
freq_df
        

,22042100.0,22083020.0,22089000.0,22041090.0,22087000.0,22084000.0,22086000.0,22041010.0,22083090.0,70132800.0,22083010.0,22085000.0,74032100.0,22082000.0,22051000.0
abacaxi,0,0,2,0,0,4,1,0,0,0,0,0,0,0,0
absolute,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
acai,0,0,34,0,0,0,3,0,0,0,0,0,0,0,0
accord,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
achocolatado,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zinf,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
zinfandel,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0
énica,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
énico,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# gera lista de 5 palavras mais frequentes em cada NCM
palavras_ncm = {}
for n in ncm:
    p_chave = []
    temp = freq_df[n]
    temp = temp.sort_values(ascending=False)
    for i in range(num_key_words):
        p_chave.append(temp.index[i])
    palavras_ncm[n] = p_chave     
palavras_ncm

{22042100.0: ['vinho', 'tinto', 'sauv', 'branco', 'caber'],
 22083020.0: ['uisque', 'anos', 'destilada', 'whisky', 'glen'],
 22089000.0: ['destilada', 'vodca', 'sabor', 'sabores', 'tradicional'],
 22041090.0: ['vinho', 'tinto', 'spumante', 'espuma', 'espumante'],
 22087000.0: ['licor', 'sabor', 'marula', 'choc', 'chocolate'],
 22084000.0: ['destilada', 'cacha', 'cachaca', 'sabor', 'cana'],
 22086000.0: ['vodca', 'destilada', 'vodka', 'sabor', 'taca'],
 22041010.0: ['champ', 'champagne', 'brut', 'vinho', 'rose'],
 22083090.0: ['uisque', 'whisky', 'ballantines', 'anos', 'walker'],
 70132800.0: ['taca', 'vinho', 'produto', 'whisky', 'destilada'],
 22083010.0: ['uisque', 'whisky', 'daniel', 'jack', 'anos'],
 22085000.0: ['destilada', 'bombay', 'hendrick', 'sapphire', 'sabor'],
 74032100.0: ['destilada', 'sabor', 'bombay', 'sapphire', 'plantas'],
 22082000.0: ['conhaque', 'tawny', 'asti', 'graham', 'port'],
 22051000.0: ['vinho', 'tinto', 'caber', 'cabernet', 'cabern']}

In [19]:
# retorna quantidade de palavras chaves na NCM atual para cada produto
def conta_key_words(ncm, des_produto):
    #if type(ncm) is not str :
        #return 0
    if palavras_ncm.get(ncm) is None:
        return 0
    cont = [p for p in palavras_ncm[ncm] if p in des_produto and rep[ncm] > ncm_representativa]
    return len(cont)


In [20]:
# inclui nova coluna em prod com quantidade de palavras chaves na descrição e NCM atual
prod["qde_key_words"] = list(map(conta_key_words, prod["COD_NCM"], prod["DES_PRODUTO"].str.lower()))

In [21]:
prod

,CATEGORIA,COD_PRODUTO,DES_PRODUTO,COD_NCM,MARCA,qde_key_words
0,20,2024724001,CONHAQUE REMY MARTIN 100CL,22082000.0,REMY MARTIN,1
1,20,2031240001,"COURVOISIER VSOP, CONHAQUE AGUARDENTE DA DESTI...",22082000.0,COURVOISIER,1
2,20,2031259001,CONHAQUE MARTELL VSOP 40% 1L,22082000.0,MARTELL,1
3,20,2034010004,MARTELL XO 40% 70CL COGNAC,22089000.0,MARTELL,0
4,20,2039594001,CONHAQUE MARTELL VS FINE CHAMP 1L 40% 1L,22082000.0,MARTELL,1
...,...,...,...,...,...,...
3328,20,6888271,NA,22083020.0,MACALLAN,0
3329,20,6888289,NA,22085000.0,BOMBAY SAPPHIRE,0
3330,20,6888297,NA,22041090.0,LOLEA,0
3331,20,6888301,NA,22041090.0,LOLEA,0


In [22]:
key_words = pd.DataFrame(palavras_ncm)
key_words

,22042100.0,22083020.0,22089000.0,22041090.0,22087000.0,22084000.0,22086000.0,22041010.0,22083090.0,70132800.0,22083010.0,22085000.0,74032100.0,22082000.0,22051000.0
0,vinho,uisque,destilada,vinho,licor,destilada,vodca,champ,uisque,taca,uisque,destilada,destilada,conhaque,vinho
1,tinto,anos,vodca,tinto,sabor,cacha,destilada,champagne,whisky,vinho,whisky,bombay,sabor,tawny,tinto
2,sauv,destilada,sabor,spumante,marula,cachaca,vodka,brut,ballantines,produto,daniel,hendrick,bombay,asti,caber
3,branco,whisky,sabores,espuma,choc,sabor,sabor,vinho,anos,whisky,jack,sapphire,sapphire,graham,cabernet
4,caber,glen,tradicional,espumante,chocolate,cana,taca,rose,walker,destilada,anos,sabor,plantas,port,cabern


In [23]:
def ncm_gol(des_produto):
    ret = []
    for p in gol.values:
        if des_produto.count(p[1]) > 0:
            ret.append(p[2])
    return ret    

In [24]:
def sugere_ncm(des_produto):
    if des_produto == "NA":
        return None
    ret = []
    busca_gol = ncm_gol(des_produto)
    if len(busca_gol) > 0:
        return busca_gol
    
    for l in range(len(key_words.columns)):
        cont = 0
        for i in range(num_key_words):
            if key_words.values[i][l] in des_produto:
                cont += 1
        ret.append(cont)     
    mx = max(ret)
    if mx == 0:
        return None
    return ncm[ret.index(mx)]

In [94]:
s = "CHOCLATE AO LEITE SEM RECHEIO APRESRETNADO EM FORMATO DE PASTILHAS EM EMBALAGEM DE PLASTICO PESANDO 250GR MMS".lower()
sugere_ncm(s)

22087000.0

In [25]:
prod["ncm_sugerida"] = list(map(sugere_ncm, prod["DES_PRODUTO"].str.lower()))

In [26]:
prod["Revisar?"] = [False if i in prod["ncm_sugerida"] else True for i in prod["COD_NCM"]]

In [27]:
def revisar(ncm, sugeridas):
    if ncm is None or sugeridas is None:
        return True

    if type(sugeridas) is list :
        if sugeridas.count(ncm) == 0:
            return True
        else:
            return False
    
    elif sugeridas == ncm:
        return False
    
    return True

In [28]:
#prod["Revisar?"] = list(map(lambda x, y : True if x is None or x not in y else False , prod["COD_NCM"], prod["ncm_sugerida"]))
prod["Revisar?"] = list(map(revisar , prod["COD_NCM"], prod["ncm_sugerida"]))

In [29]:
prod["COD_NCM"].isnull().sum()

2

In [30]:
prod

,CATEGORIA,COD_PRODUTO,DES_PRODUTO,COD_NCM,MARCA,qde_key_words,ncm_sugerida,Revisar?
0,20,2024724001,CONHAQUE REMY MARTIN 100CL,22082000.0,REMY MARTIN,1,[22082000],False
1,20,2031240001,"COURVOISIER VSOP, CONHAQUE AGUARDENTE DA DESTI...",22082000.0,COURVOISIER,1,"[22042100, 22082000]",False
2,20,2031259001,CONHAQUE MARTELL VSOP 40% 1L,22082000.0,MARTELL,1,[22082000],False
3,20,2034010004,MARTELL XO 40% 70CL COGNAC,22089000.0,MARTELL,0,[22082000],True
4,20,2039594001,CONHAQUE MARTELL VS FINE CHAMP 1L 40% 1L,22082000.0,MARTELL,1,[22082000],False
...,...,...,...,...,...,...,...,...
3328,20,6888271,NA,22083020.0,MACALLAN,0,None,True
3329,20,6888289,NA,22085000.0,BOMBAY SAPPHIRE,0,None,True
3330,20,6888297,NA,22041090.0,LOLEA,0,None,True
3331,20,6888301,NA,22041090.0,LOLEA,0,None,True


In [31]:
# grava prod em arquivo Excel
with pd.ExcelWriter(arquivo[: arquivo.find('.')]+ str(categoria) + '_output.xlsx') as writer:
    prod.to_excel(writer, sheet_name = 'Produtos')
    key_words.to_excel(writer, sheet_name = 'Palavras Chaves')
    freq_df.to_excel(writer, sheet_name = 'Palavras Completo')
    rep.to_excel(writer, sheet_name = 'Representatividade')